In [1]:
import pandas as pd
import numpy as np

In [2]:
# Lê o arquivo CSV
df = pd.read_csv('diabetes_prediction_dataset.csv')
# Mostra as 5 primeiras linhas da tabela
df.head()


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [ ]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer

    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """

    np.random.seed()

    W1 = np.random.randn(n_h, n_x) * 0.001
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.001
    b2 = np.zeros((n_y, 1))

    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}

    return parameters

In [ ]:
def feed_forward(X, parameters):
    """
    Argument:
    X -- dados de input de tamanho  (n_x, m)
    parameters -- Dicionario python com nossos parametros (saída da função de inicialização)

    Returns:
    A2 -- A saída sigmóide da segunda ativação
    cache -- um dicionário contendo "Z1", "A1", "Z2" e "A2"
    """

    Z1 = np.dot(parameters["W1"], X) + parameters["b1"]
    A1 = np.tanh(Z1)
    Z2 = np.dot(parameters["W2"], A1) + parameters["b2"]
    A2 = sigmoid(Z2)

    assert (A2.shape == (1, X.shape[1]))

    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}

    return A2, cache

In [ ]:
def compute_cost(A2, Y, parameters):
    """
    Calcula o custo de entropia cruzada dado na equação (13)

    Arguments:
    A2 - A saída sigmóide da segunda ativação, de formato (1, número de observações)
    Y - vetor de forma de rótulos "verdadeiros" (1, número de observações)
    parâmetros - dicionário python contendo seus parâmetros W1, b1, W2 e b2

    Returns:
    cost -- custo de entropia cruzada dada a equação (13)
    """

    m = Y.shape[1]

    logprobs = np.multiply(np.log(A2), Y)
    cost = - np.sum(logprobs)

    cost = np.squeeze(cost)
    assert (isinstance(cost, float))

    return cost

In [ ]:
def backward_propagation(parameters, cache, X, Y):
    """
    Implemente a backward propagation usando as instruções acima.

    Arguments:
     parameters - dicionário python contendo os parâmetros
     cache -- um dicionário contendo "Z1", "A1", "Z2" e "A2".
     X – dados de entrada da forma (2, número de observações)
     Y - vetor de forma de rótulos "verdadeiros" (1, número de observações)

    Returns:
    grads -- dicionário python contendo seus gradientes em relação a diferentes parâmetros
    """
    m = X.shape[1]

    dZ2 = cache["A2"] - Y
    dW2 = (1 / m) * np.dot(dZ2, cache["A1"].T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.dot(parameters["W2"].T, dZ2) * (1 - np.power(cache["A1"], 2))
    dW1 = (1 / m) * np.dot(dZ1, X.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)

    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}

    return grads

In [ ]:
def layer_sizes(X, Y):
    """
    Arguments:
    X – conjunto de dados de entrada de forma (tamanho de entrada, número de observações)
    Y – rótulos de forma (tamanho de saída, número de observações)

    Returns:
    n_x – o tamanho da camada de entrada
    n_h – o tamanho da camada oculta
    n_y – o tamanho da camada de saída
    """
    n_x = X.shape[0]
    n_h = 4
    n_y = Y.shape[0]
    return (n_x, n_h, n_y)

In [ ]:
def update_parameters(parameters, grads, learning_rate=1.2):
    """
    Atualiza os parâmetros usando a regra de atualização de gradiente descendente

    Arguments:
    parameters – dicionário python contendo seus parâmetros
    grads – dicionário python contendo seus gradientes

    Returns:
    parameters – dicionário python contendo seus parâmetros atualizados
    """
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']

    W1 = W1 - (learning_rate * grads["dW1"])
    b1 = b1 - (learning_rate * grads["db1"])
    W2 = W2 - (learning_rate * grads["dW2"])
    b2 = b2 - (learning_rate * grads["db2"])

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}

    return parameters

In [ ]:
def sigmoid(x):
    """
    Calcular o sigmóide de x

    Arguments:
    x -- Uma matriz escalar ou numpy de qualquer tamanho.

    Return:
    s -- sigmoid(x)
    """
    s = 1/(1+np.exp(-x))
    return s

In [ ]:
def nn_model(X, Y, n_h, num_iterations=10000, print_cost=False):
    """
    Arguments:
    X - conjunto de dados de forma (2, número de observações)
    Y - rótulos de forma (1, número de observações)
    n_h – tamanho da camada oculta
    num_iterations -- Número de iterações no loop de descida gradiente
    print_cost -- se True, imprime o custo a cada 1000 iterações

    Returns:
    parameters -- parâmetros aprendidos pelo modelo. Eles podem então ser usados para prever.
    """

    np.random.seed()
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]

    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']

    for i in range(0, num_iterations):
        A2, cache = feed_forward(X, parameters)

        cost = compute_cost(A2, Y, parameters)

        grads = backward_propagation(parameters, cache, X, Y)

        parameters = update_parameters(parameters, grads, learning_rate=1.2)

        if print_cost and i % 1000 == 0:
            print("Custo após a interação %i: %f" % (i, cost))

    return parameters